[Versión Interactiva](https://mybinder.org/v2/git/https%3A%2F%2Fgithub.com%2Fjuandadamo%2Fclases_fiuba/HEAD?filepath=6731%2Fconduccion-unidades-problema.ipynb)

In [1]:
%%capture
!pip install ht pint numpy matplotlib Coolprop xlrd xlwt meteocalc pint

In [267]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd
import ht,pint
import CoolProp as cp
import matplotlib.pyplot as plt
from IPython.display import  Latex
import meteocalc
ureg = pint.UnitRegistry()#(fmt_locale='es_ES')
from scipy.optimize import brentq

In [268]:
Diametros_nominales = np.array([1,1.25,1.5,2,2.5,3,3.5,4])
Diam_exterior = np.array([33.4,42.2,48.3,60.3,73,88.9,101.6,114.3])
espesores_caños = np.array([3.38,3.56,3.68,3.91,5.16,5.49,5.74,6.02])
Diam_interior = Diam_exterior - 2*espesores_caños

In [269]:
Temp_f = np.array([5,-5,-15,-25,-35,-45])
espesores_aislacion = np.array([0,25,40,50,65,75,90,100,115,125])
k_aislacion = np.array([0.039,0.038,0.027,0.032])
hconv_fl_int = 850
hconv_aire_ext_i = 4
hconv_aire_ext_e = 34
#Aire exterior
Temp_aire_int = 32
Temp_aire_ext = 36
Humedad_aire_int = 80
Humedad_aire_ext = 94
espesor_recubrimiento = 0.5
k_recubrimiento = 0.5
emisividad_recubrimiento = 0.1
sigma_SB = 5.67e-8

In [270]:
Qmax = 25
T_rocio_aire_ext  = meteocalc.dew_point(Temp_aire_ext,Humedad_aire_ext).value
T_rocio_aire_int  = meteocalc.dew_point(Temp_aire_int,Humedad_aire_int).value
Tmax_1 = T_rocio_aire_ext + 0
Tmax_2 = T_rocio_aire_int + 0

In [271]:
def Resistencia_termica(hi,he,R1,R2,R3, R4,k1,k2,k3,hrad):
    S1,S2,S3,S4 = [2*np.pi*Ri for Ri in [R1,R2,R3,R4]]
    Rconv1 = 1/hi/(S1)
    Rconv2 = 1/(he*S4)
    R_caño = np.log(R2/R1)/(k1*2*np.pi)
    R_rec = np.log(R4/R3)/(k2*2*np.pi)
    R_aisl = np.log(R3/R2)/(k3*2*np.pi)
    R_rad =  1/(hrad*S4)
    R_serie =  Rconv1+ +R_caño +R_rec +R_aisl
    return R_serie,Rconv1,Rconv2,R_aisl,R_caño,R_rec,R_rad

In [277]:
#j = 4 #Temperatura del fluido
#i = 0  #Diametro Interno
#k = 1  #espesor aislacion

def deltaT(T4,k_aisla,Temp_fluido,espesor_aislacion,Diametro_interior,espesor_caños):
    Tmedia = (T4+Temp_aire_ext)/2+273
    hrad = Tmedia**3*sigma_SB*emisividad_recubrimiento
    R1 = Diametro_interior/2*1e-3
    R2 = R1+espesor_caños*1e-3
    R3 = R2 + espesor_aislacion*1e-3
    R4 = R3 + espesor_recubrimiento*1e-3
    material = ht.nearest_material('steel')
    k_acero = ht.k_material(material)
    k_recubrimiento = 0.5
    #k_aisla = k_aislacion[k]
    R_serie,Rconv1,Rconv2,R_aisl,R_caño,R_rec,R_rad = Resistencia_termica(hconv_fl_int,hconv_aire_ext_e,R1,R2,R3,R4,k_acero,k_recubrimiento,k_aisla,hrad)
    Rparalelo = (1/Rconv2+1/R_rad)**-1
    R_total = R_serie + Rparalelo 
    Qreal = (Temp_aire_ext - Temp_fluido)/R_total
    #print(f'Q={Qreal:.1f}')
    T4b = Temp_fluido+Qreal*R_serie             
    #print(T4b,Qreal)
    #print(Rparalelo)
    return T4b - T4
def calor(T4,k_aisla,Temp_fluido,espesor_aislacion,Diametro_interior,espesor_caños):

    Tmedia = (T4+Temp_aire_ext)/2+273

    hrad = Tmedia**3*sigma_SB*emisividad_recubrimiento

    R1 = Diametro_interior/2*1e-3
    R2 = R1+espesor_caños*1e-3
    R3 = R2 + espesor_aislacion*1e-3
    R4 = R3 + espesor_recubrimiento*1e-3
    material = ht.nearest_material('steel')
    k_acero = ht.k_material(material)
    k_recubrimiento = 0.5
    #k_aisla = k_aislacion[k]
    R_serie,Rconv1,Rconv2,R_aisl,R_caño,R_rec,R_rad = Resistencia_termica(hconv_fl_int,hconv_aire_ext_e,R1,R2,R3,R4,k_acero,k_recubrimiento,k_aisla,hrad)
    Rparalelo = (1/Rconv2+1/R_rad)**-1
    R_total = R_serie + Rparalelo 
    Qreal = (Temp_aire_ext - Temp_fluido)/R_total

    return Qreal

In [278]:
nombres_columnas = [str(Temp_i) for Temp_i in Temp_f]
nombres_columnas.insert(0,'Tamaño Nominal caño')
A = pd.DataFrame(columns=nombres_columnas)
B = pd.DataFrame(columns=nombres_columnas)
C = pd.DataFrame(columns=nombres_columnas)

In [279]:
T4min,T4max = (-10,Temp_aire_ext)
T4final = 0
Qtotal = 1.1*Qmax
k_aisla = k_aislacion[0]

for i in range(8):
    A.loc[i] = 0
    B.loc[i] = 0
    C.loc[i] = 0
    A.iloc[i,0] = Diam_interior[i]
    B.iloc[i,0] = Diam_interior[i]
    C.iloc[i,0] = Diam_interior[i]
    for it,Temp_fluido in enumerate(Temp_f[:]):
        T4final = (Temp_fluido+Temp_aire_ext)/2
        k = 0
        while np.logical_or(Qtotal>Qmax,T4final< Tmax_1):
            espesores_aisla = espesores_aislacion[k]
            deltaT_min = deltaT(T4min,k_aisla,Temp_fluido,espesores_aisla,Diam_interior[i],espesores_caños[i])
            deltaT_max = deltaT(T4max,k_aisla,Temp_fluido,espesores_aisla,Diam_interior[i],espesores_caños[i])
            if np.sign(deltaT_min*deltaT_max) ==1:
                k += 1
                espesores_aisla = espesores_aislacion[k]
            else:
                T4final = brentq(deltaT, T4min,T4max,(k_aisla,Temp_fluido,espesores_aisla,Diam_interior[i],espesores_caños[i]))  
                Qtotal = calor(T4final,k_aisla,Temp_fluido,espesores_aisla,Diam_interior[i],espesores_caños[i])  
                k += 1
        A.iloc[i,it+1]= espesores_aisla
        B.iloc[i,it+1]= np.around(T4final,1)
        C.iloc[i,it+1]= np.around(Qtotal,1)

In [280]:
A

,Tamaño Nominal caño,5,-5,-15,-25,-35,-45
0,26.64,25,40,40,40,50,50
1,35.08,25,40,40,40,50,50
2,40.94,25,40,40,40,50,50
3,52.48,25,40,40,50,50,65
4,62.68,25,40,40,50,50,65
5,77.92,25,40,40,50,50,65
6,90.12,25,40,40,50,65,65
7,102.26,40,40,40,50,65,75


In [262]:
B

,Tamaño Nominal caño,5,-5,-15,-25,-35,-45
0,26.64,35.1,35.3,35.2,35.0,35.1,35.0
1,35.08,35.1,35.3,35.1,35.0,35.1,35.0
2,40.94,35.0,35.3,35.1,34.9,35.0,34.9
3,52.48,35.0,35.2,35.0,35.1,35.0,35.2
4,62.68,34.9,35.2,35.0,35.1,34.9,35.1
5,77.92,34.9,35.2,35.0,35.0,34.9,35.1
6,90.12,34.9,35.1,34.9,35.0,35.2,35.0
7,102.26,35.3,35.1,34.9,35.0,35.1,35.2


In [263]:
C

,Tamaño Nominal caño,5,-5,-15,-25,-35,-45
0,26.64,8.0,8.1,10.0,12.0,12.4,14.1
1,35.08,9.4,9.3,11.5,13.8,14.1,16.1
2,40.94,10.3,10.1,12.5,15.0,15.3,17.4
3,52.48,12.1,11.6,14.5,15.0,17.5,17.1
4,62.68,14.0,13.3,16.5,17.0,19.8,19.2
5,77.92,16.4,15.3,19.0,19.5,22.7,21.7
6,90.12,18.2,16.9,21.0,21.4,20.8,23.8
7,102.26,14.0,18.5,23.0,23.3,22.6,23.4


In [87]:
espesores_aislacion

array([  0,  25,  40,  50,  65,  75,  90, 100, 115, 125])